In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install datasets

     |████████████████████████████████| 4.0 MB 5.0 MB/s 
     |████████████████████████████████| 596 kB 56.1 MB/s 
     |████████████████████████████████| 77 kB 7.2 MB/s 
     |████████████████████████████████| 895 kB 57.8 MB/s 
     |████████████████████████████████| 6.5 MB 33.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 4.9 MB/s 
     |████████████████████████████████| 212 kB 63.2 MB/s 
     |████████████████████████████████| 136 kB 57.3 MB/s 
     |████████████████████████████████| 1.1 MB 62.0 MB/s 
     |████████████████████████████████| 127 kB 60.4 MB/s 
     |████████████████████████████████| 271 kB 60.1 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [3]:
import os
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, logging
import pandas as pd 
import numpy as np  
import ast
from IPython.display import clear_output
from datasets import Dataset, load_metric
import torch
import multiprocessing

In [4]:
authors = ['Ataol Behramoğlu', 'Attila İlhan', 'Cahit Sıtkı Tarancı', 'Cahit Zarifoğlu',
 'Can Yücel', 'Cemal Safi', 'Cemal Süreya', 'Edip Cansever',
 'Mehmet Akif Ersoy', 'Nazım Hikmet Ran', 'Necip Fazıl Kısakürek',
 'Orhan Veli Kanık', 'Turgut Uyar', 'Özdemir Asaf', 'Ümit Yaşar Oğuzcan']

In [5]:
test_dataset = pd.read_csv("/content/drive/MyDrive/test_dataset.csv",sep=";")
for column in ["input_ids","token_type_ids","attention_mask"]:
    for i in range(0,len(test_dataset[column])):
        a = torch.IntTensor([ast.literal_eval(test_dataset[column][i])])
        test_dataset[column][i] = a

for i in range(0,len(test_dataset["label"])):
    test_dataset["label"][i] = np.array(test_dataset["label"][i])


test_label = test_dataset.pop("label")

clear_output()

In [6]:
test_dict = test_dataset.to_dict('records')
test_dataset = pd.read_csv("/content/drive/MyDrive/test_dataset.csv",sep=";")
for column in ["input_ids","token_type_ids","attention_mask"]:
    for i in range(0,len(test_dataset[column])):
        a = ast.literal_eval(test_dataset[column][i])
        test_dataset[column][i] = a

test_ds = Dataset.from_pandas(test_dataset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")

for model_path in os.scandir("/content/drive/MyDrive/ModelTakeTwo"):
    if model_path.is_dir():
        path = model_path.path.replace("\\","/")
        trainer = Trainer(
          model = BertForSequenceClassification.from_pretrained(path),
          eval_dataset = test_ds
        )
        predictions = trainer.predict(test_ds)
        equality_array = []
        for i in range(0,len(predictions[0])):
          equality_array.append(np.argmax(predictions[0][i])==test_ds[i]["label"])

        print(f"model's folder: {path} accuracy: {np.array(equality_array).mean()}")


Downloading:   0%|          | 0.00/972k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

***** Running Prediction *****
  Num examples = 10336
  Batch size = 8


loading configuration file /content/drive/MyDrive/ModelTakeTwo/checkpoint-2000/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/model/last",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4

...

All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/ModelTakeTwo/checkpoint-77000.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 10336
  Batch size = 8


loading configuration file /content/drive/MyDrive/ModelTakeTwo/last/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/model/last",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
    "LABEL_13": 13,
    "LABEL_14": 14,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABE

model's folder: /content/drive/MyDrive/ModelTakeTwo/checkpoint-77000 accuracy: 0.5163506191950464


All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/ModelTakeTwo/last.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 10336
  Batch size = 8


model's folder: /content/drive/MyDrive/ModelTakeTwo/last accuracy: 0.5153831269349846


In [ ]:
#The most accurate AI is checkpoint-7300